In [29]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-hhxl8f4p
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-hhxl8f4p
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   


In [31]:
%cd /root/text-to-mesh

/root/text-to-mesh


In [32]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,    
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

pkg = torch.load("./MeshGPT-autoencoder_0.38_x1s.pt") 
autoencoder.load_state_dict(pkg['model'])
for param in autoencoder.parameters():
    param.requires_grad = True

In [45]:
dataset = MeshDataset.load("./objverse_250f_45.9M_3086_labels_53730_10_min_x1_aug.npz")  
dataset2 = MeshDataset.load("./objverse_250f_229.7M_3086_labels_268650_10_min_x5_aug.npz")
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
dataset.data.extend(dataset2.data) 
dataset.sort_dataset_keys()
print("length", len(dataset.data))

def format_value(value):
    if value >= 1_000_000_000:
        return f"{value / 1_000_000_000:.1f}B"
    elif value >= 1_000_000:
        return f"{value / 1_000_000:.1f}M"
    else:
        return f"{value}"

tokens = 0
for item in dataset.data:
    tokens += len(item['faces']) * 6 
total_tokens = format_value(tokens)
print("Tokens:", total_tokens)

[MeshDataset] Loaded 53730 entrys
[MeshDataset] Created from 53730 entrys
[MeshDataset] Loaded 268650 entrys
[MeshDataset] Created from 268650 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 21560 entrys
[MeshDataset] Created from 21560 entrys
length 346096
Tokens: 299.7M


In [42]:
pkg = torch.load("./MeshGPT-autoencoder_0.38_x1s.pt") 
autoencoder.load_state_dict(pkg['model'])
for param in autoencoder.parameters():
    param.requires_grad = True

In [43]:
batch_size=16 # The batch size should be max 64.
grad_accum_every =4
# So set the maximal batch size (max 64) that your VRAM can handle and then use grad_accum_every to create a effective batch size of 64, e.g  16 * 4 = 64
learning_rate = 1e-3 # Start with 1e-3 then at stagnation around 0.35, you can lower it to 1e-4.

autoencoder.commit_loss_weight = 0.2 # Set dependant on the dataset size, on smaller datasets, 0.1 is fine, otherwise try from 0.25 to 0.4.
autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                             batch_size=batch_size,
                                             grad_accum_every = grad_accum_every,
                                             learning_rate = learning_rate,
                                             checkpoint_every_epoch=5)
 
loss = autoencoder_trainer.train(480, diplay_graph= True)   

Epoch 1/480:   0%|          | 1/4840 [00:00<10:09,  7.94it/s, commit_loss=-0.625, loss=0.226, recon_loss=0.383]

Epoch 1/480: 100%|██████████| 4840/4840 [10:19<00:00,  7.81it/s, commit_loss=-0.616, loss=0.19, recon_loss=0.344]   


Epoch 1 average loss: 0.19048658090620493 recon loss: 0.3500: commit_loss -0.6382


Epoch 2/480: 100%|██████████| 4840/4840 [09:57<00:00,  8.10it/s, commit_loss=-0.627, loss=0.187, recon_loss=0.344]


Epoch 2 average loss: 0.1835588489189621 recon loss: 0.3471: commit_loss -0.6543


...

Epoch 5 average loss: 0.17643269283835553 recon loss: 0.3445: commit_loss -0.6724          avg loss speed: 0.005314637818078699 epochs left: 23.25


Epoch 6/480: 100%|██████████| 4840/4840 [09:58<00:00,  8.08it/s, commit_loss=-0.557, loss=0.214, recon_loss=0.353] 


Epoch 6 average loss: 0.174829849699312 recon loss: 0.3439: commit_loss -0.6763          avg loss speed: 0.004542095596920009 epochs left: 27.56


Epoch 7/480:  73%|███████▎  | 3542/4840 [07:15<02:39,  8.13it/s, commit_loss=-0.724, loss=0.161, recon_loss=0.342] 


KeyboardInterrupt: 

In [44]:

pkg = dict( model = autoencoder.state_dict(), ) 
torch.save(pkg, str(f"./MeshGPT-autoencoder_0.38_x1s.pt"))
import torch
import random
from tqdm import tqdm 
from meshgpt_pytorch import mesh_render 

min_mse, max_mse = float('inf'), float('-inf')
min_coords, min_orgs, max_coords, max_orgs = None, None, None, None
random_samples, random_samples_pred, all_random_samples = [], [], []
total_mse, sample_size = 0.0, 200

random.shuffle(dataset.data)
autoencoder = autoencoder.to("cuda")
for item in tqdm(dataset.data[:sample_size]):
    item['faces'] = item['faces'].to("cuda")
    item['vertices'] = item['vertices'].to("cuda")
    item['face_edges'] = item['face_edges'].to("cuda")
    codes = autoencoder.tokenize(vertices=item['vertices'], faces=item['faces'], face_edges=item['face_edges']) 
    
    codes = codes.flatten().unsqueeze(0)
    codes = codes[:, :codes.shape[-1] // autoencoder.num_quantizers * autoencoder.num_quantizers] 
 
    coords, mask = autoencoder.decode_from_codes_to_faces(codes)
    orgs = item['vertices'][item['faces']].unsqueeze(0)

    mse = torch.mean((orgs.view(-1, 3).cpu() - coords.view(-1, 3).cpu())**2)
    total_mse += mse 

    if mse < min_mse: min_mse, min_coords, min_orgs = mse, coords, orgs
    if mse > max_mse: max_mse, max_coords, max_orgs = mse, coords, orgs
 
    if len(random_samples) <= 30:
        random_samples.append((coords, mask)) 
    else:
        all_random_samples.extend([ random_samples])
        random_samples, random_samples_pred = [], []

print(f'MSE AVG: {total_mse / sample_size:.10f}, Min: {min_mse:.10f}, Max: {max_mse:.10f}')    
mesh_render.save_rendering(f'./mse_rows.obj', all_random_samples)

 

100%|██████████| 200/200 [00:11<00:00, 16.79it/s]


MSE AVG: 0.0002693227, Min: 0.0000152457, Max: 0.0249535665
[Save_rendering] Saved at ./mse_rows.obj


In [ ]:
transformer.conditioner.text_models[0].model.to("cuda")
transformer = transformer.to("cuda")

dataset.embed_texts(transformer,1)

In [ ]:
 
torch.cuda.empty_cache()
gc.collect()  
 
batch_size = 2
grad_accum_every =4     
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                  accelerator_kwargs = {"mixed_precision" : "fp16"}, optimizer_kwargs = { "eps": 1e-7} , 
                                 learning_rate = 1e-3, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(35, stop_at_loss = 0.00005)   
 

In [ ]:
# pkg = dict( model = autoencoder.state_dict(), ) 
# torch.save(pkg, str(f"./MeshGPT-autoencoder_0.38_x1s.pt"))

In [ ]:
import torch
import random
from tqdm import tqdm 
from meshgpt_pytorch import mesh_render 

min_mse, max_mse = float('inf'), float('-inf')
min_coords, min_orgs, max_coords, max_orgs = None, None, None, None
random_samples, random_samples_pred, all_random_samples = [], [], []
total_mse, sample_size = 0.0, 200

random.shuffle(dataset.data)
autoencoder = autoencoder.to("cuda")
for item in tqdm(dataset.data[:sample_size]):
    item['faces'] = item['faces'].to("cuda")
    item['vertices'] = item['vertices'].to("cuda")
    item['face_edges'] = item['face_edges'].to("cuda")
    codes = autoencoder.tokenize(vertices=item['vertices'], faces=item['faces'], face_edges=item['face_edges']) 

    codes = codes.flatten().unsqueeze(0)
    codes = codes[:, :codes.shape[-1] // autoencoder.num_quantizers * autoencoder.num_quantizers] 
 
    coords, mask = autoencoder.decode_from_codes_to_faces(codes)
    orgs = item['vertices'][item['faces']].unsqueeze(0)

    mse = torch.mean((orgs.view(-1, 3).cpu() - coords.view(-1, 3).cpu())**2)
    total_mse += mse 

    if mse < min_mse: min_mse, min_coords, min_orgs = mse, coords, orgs
    if mse > max_mse: max_mse, max_coords, max_orgs = mse, coords, orgs
 
    if len(random_samples) <= 30:
        random_samples.append((coords, mask)) 
    else:
        all_random_samples.extend([ random_samples])
        random_samples, random_samples_pred = [], []

print(f'MSE AVG: {total_mse / sample_size:.10f}, Min: {min_mse:.10f}, Max: {max_mse:.10f}')
mesh_render.save_rendering(f'.\mse_rows.obj', all_random_samples)

 59%|█████▉    | 118/200 [00:07<00:05, 15.11it/s]


KeyboardInterrupt: 

In [ ]:
#pkg = dict( model = autoencoder.state_dict(), ) 
#torch.save(pkg, str(f"./MeshGPT-autoencoder_testts.pt"))